# JSONLD

In [8]:
import json
from pyld import jsonld
from rdflib import Graph


## @context
JSON-LD must have a `@context`. In JSON-LD, the `@context` is used to define the meaning of terms used in the document. The `@context` can be specified either as a JSON object or as a URL that points to an external context definition. 

- `@context` as JSON object -> mapping via external vocabulary.
- `@context` as URL link -> mapping via external context.

When specified as a JSON object, the `@context` defines a mapping between the keys in the JSON document and their corresponding IRIs (Internationalized Resource Identifiers) in an external vocabulary. When specified as a URL, the `@context` points to an external context definition that contains the mapping information. Specifying the `@context` as a JSON object allows you to define the context directly within the document, while specifying it as a URL allows you to reuse the same context across multiple documents.


Here is an example of specifying the `@context` as a JSON object:

```json
{
    "@context": {
        "name": "http://schema.org/name",
        "jobTitle": "http://schema.org/jobTitle"
    },
    "name": "Jane Doe",
    "jobTitle": "Software Engineer"
}
```

In this example, the `@context` is defined as a JSON object that maps the keys `name` and `jobTitle` to their corresponding IRIs in the `http://schema.org/` vocabulary.

Alternatively, the `@context` can be specified as a URL that points to an external context definition. Here is an example:

```json
{
    "@context": "https://example.com/my-context.jsonld",
    "name": "Jane Doe",
    "jobTitle": "Software Engineer"
}
```

In this example, the `@context` is specified as a URL that points to an external context definition located at `https://example.com/my-context.jsonld`. This external context definition should contain a JSON-LD document that defines the mapping.

## Example 2 sample from W3C
- https://www.w3.org/TR/json-ld11-api/#example-2-sample-json-ld-document

In [9]:
doc = {
    "@context": {
        "name": "http://xmlns.com/foaf/0.1/name",
        "knows": "http://xmlns.com/foaf/0.1/knows"
    },
    "@id": "http://me.markus-lanthaler.com/",
    "name": "Markus Lanthaler",
    "knows": [
        {
            "name": "Dave Longley"
        }
    ]
}

In [10]:
doc["@context"]

{'name': 'http://xmlns.com/foaf/0.1/name',
 'knows': 'http://xmlns.com/foaf/0.1/knows'}

## Create JSON-LD input

In [11]:
json_ld_input = json.dumps(doc, indent=2)
print(json_ld_input)

{
  "@context": {
    "name": "http://xmlns.com/foaf/0.1/name",
    "knows": "http://xmlns.com/foaf/0.1/knows"
  },
  "@id": "http://me.markus-lanthaler.com/",
  "name": "Markus Lanthaler",
  "knows": [
    {
      "name": "Dave Longley"
    }
  ]
}


## JSON-LD expansion/compaction operation
### Compact

In [12]:
compacted_data = jsonld.compact(doc, doc["@context"])
print(json.dumps(compacted_data, indent=2))

{
  "@context": {
    "name": "http://xmlns.com/foaf/0.1/name",
    "knows": "http://xmlns.com/foaf/0.1/knows"
  },
  "@id": "http://me.markus-lanthaler.com/",
  "knows": {
    "name": "Dave Longley"
  },
  "name": "Markus Lanthaler"
}


### Expand

In [13]:
# expand on doc
expanded_data = jsonld.expand(doc)
print(json.dumps(expanded_data, indent=2))

[
  {
    "@id": "http://me.markus-lanthaler.com/",
    "http://xmlns.com/foaf/0.1/knows": [
      {
        "http://xmlns.com/foaf/0.1/name": [
          {
            "@value": "Dave Longley"
          }
        ]
      }
    ],
    "http://xmlns.com/foaf/0.1/name": [
      {
        "@value": "Markus Lanthaler"
      }
    ]
  }
]


In [14]:
# expand on compact
expanded_data = jsonld.expand(compacted_data)
print(json.dumps(expanded_data, indent=2))

[
  {
    "@id": "http://me.markus-lanthaler.com/",
    "http://xmlns.com/foaf/0.1/knows": [
      {
        "http://xmlns.com/foaf/0.1/name": [
          {
            "@value": "Dave Longley"
          }
        ]
      }
    ],
    "http://xmlns.com/foaf/0.1/name": [
      {
        "@value": "Markus Lanthaler"
      }
    ]
  }
]


### Flatten

In [15]:
flattened = jsonld.flatten(doc)
print(json.dumps(flattened, indent=2))

[
  {
    "@id": "_:b0",
    "http://xmlns.com/foaf/0.1/name": [
      {
        "@value": "Dave Longley"
      }
    ]
  },
  {
    "@id": "http://me.markus-lanthaler.com/",
    "http://xmlns.com/foaf/0.1/knows": [
      {
        "@id": "_:b0"
      }
    ],
    "http://xmlns.com/foaf/0.1/name": [
      {
        "@value": "Markus Lanthaler"
      }
    ]
  }
]


### RDF

In [16]:
jsonld.to_rdf(doc)

{'@default': [{'subject': {'type': 'blank node', 'value': '_:b0'},
   'predicate': {'type': 'IRI', 'value': 'http://xmlns.com/foaf/0.1/name'},
   'object': {'type': 'literal',
    'value': 'Dave Longley',
    'datatype': 'http://www.w3.org/2001/XMLSchema#string'}},
  {'subject': {'type': 'IRI', 'value': 'http://me.markus-lanthaler.com/'},
   'predicate': {'type': 'IRI', 'value': 'http://xmlns.com/foaf/0.1/knows'},
   'object': {'type': 'blank node', 'value': '_:b0'}},
  {'subject': {'type': 'IRI', 'value': 'http://me.markus-lanthaler.com/'},
   'predicate': {'type': 'IRI', 'value': 'http://xmlns.com/foaf/0.1/name'},
   'object': {'type': 'literal',
    'value': 'Markus Lanthaler',
    'datatype': 'http://www.w3.org/2001/XMLSchema#string'}}]}

In [17]:
json_data = json.dumps(doc, indent=2)
g = Graph().parse(data=json_data, format='json-ld')
print(g.serialize(format='json-ld', auto_compact=True, indent=4))

{
    "@context": {
        "brick": "https://brickschema.org/schema/Brick#",
        "csvw": "http://www.w3.org/ns/csvw#",
        "dc": "http://purl.org/dc/elements/1.1/",
        "dcam": "http://purl.org/dc/dcam/",
        "dcat": "http://www.w3.org/ns/dcat#",
        "dcmitype": "http://purl.org/dc/dcmitype/",
        "dcterms": "http://purl.org/dc/terms/",
        "doap": "http://usefulinc.com/ns/doap#",
        "foaf": "http://xmlns.com/foaf/0.1/",
        "geo": "http://www.opengis.net/ont/geosparql#",
        "odrl": "http://www.w3.org/ns/odrl/2/",
        "org": "http://www.w3.org/ns/org#",
        "owl": "http://www.w3.org/2002/07/owl#",
        "prof": "http://www.w3.org/ns/dx/prof/",
        "prov": "http://www.w3.org/ns/prov#",
        "qb": "http://purl.org/linked-data/cube#",
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "schema": "https://schema.org/",
        "sh": "http://www.w3.org/ns/sh